In [2]:
data = 'https://archive.ics.uci.edu/static/public/222/bank+marketing.zip'

In [3]:
!wget $data

--2024-10-09 18:24:00--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip’

bank+marketing.zip      [    <=>             ] 999.85K  1.04MB/s    in 0.9s    

2024-10-09 18:24:03 (1.04 MB/s) - ‘bank+marketing.zip’ saved [1023843]



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('bank-full.csv', sep=';')

In [3]:
df = df[['age', 'job', 'marital', 'education', 'balance', 'housing',
       'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y']]

In [4]:
df.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

In [5]:
df.mode().T

,0
age,32
job,blue-collar
marital,married
education,secondary
balance,0
housing,yes
contact,cellular
day,20
month,may
duration,124


### Question 2

In [6]:
vars = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']

In [7]:
df[vars].corrwith(df.y).abs()

ValueError: could not convert string to float: 'no'

In [ ]:
v = ['age']
df[v].corrwith(df.balance).abs()

age    0.097783
dtype: float64

In [ ]:
v = ['day']
df[v].corrwith(df.campaign).abs()

day    0.16249
dtype: float64

In [10]:
df[v].corrwith(df.pdays).abs()

day    0.093044
dtype: float64

In [11]:
v=['pdays']
df[v].corrwith(df.previous).abs()

pdays    0.45482
dtype: float64

### Question 3

In [ ]:
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [13]:
df.y = df.y.replace({'yes': 1, 'no':0})

/tmp/ipykernel_2794/2032539318.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.y = df.y.replace({'yes': 1, 'no':0})


In [ ]:
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [18]:
len(df_train), len(df_val), len(df_test)

(27126, 9042, 9043)

In [19]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [20]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

In [21]:
del df_train['y']
del df_val['y']
del df_test['y']

In [22]:
from sklearn.metrics import mutual_info_score

In [23]:
mutual_info_score(df_full_train.y, df_full_train.contact).round(2)

np.float64(0.01)

In [24]:
mutual_info_score(df_full_train.y, df_full_train.education).round(2)

np.float64(0.0)

In [25]:
mutual_info_score(df_full_train.y, df_full_train.housing).round(2)

np.float64(0.01)

In [26]:
mutual_info_score(df_full_train.y, df_full_train.poutcome).round(2)

np.float64(0.03)

### Question 4

In [27]:
df.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y             int64
dtype: object

In [30]:
categorical = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
numerical = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']

In [31]:
from sklearn.feature_extraction import DictVectorizer

In [32]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [33]:
dv = DictVectorizer(sparse=False)

In [34]:
X_train = dv.fit_transform(train_dicts)

In [35]:
X_train.shape

(27126, 46)

In [36]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [37]:
X_val = dv.transform(val_dicts)

In [38]:
dv.get_feature_names_out()

array(['age', 'balance', 'campaign', 'contact=cellular',
       'contact=telephone', 'contact=unknown', 'day', 'education=primary',
       'education=secondary', 'education=tertiary', 'education=unknown',
       'housing=no', 'housing=yes', 'job=admin.', 'job=blue-collar',
       'job=entrepreneur', 'job=housemaid', 'job=management',
       'job=retired', 'job=self-employed', 'job=services', 'job=student',
       'job=technician', 'job=unemployed', 'job=unknown',
       'marital=divorced', 'marital=married', 'marital=single',
       'month=apr', 'month=aug', 'month=dec', 'month=feb', 'month=jan',
       'month=jul', 'month=jun', 'month=mar', 'month=may', 'month=nov',
       'month=oct', 'month=sep', 'pdays', 'poutcome=failure',
       'poutcome=other', 'poutcome=success', 'poutcome=unknown',
       'previous'], dtype=object)

In [39]:
from sklearn.linear_model import LogisticRegression

In [40]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [41]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [42]:
y_pred = model.predict_proba(X_val)[:, 1]

In [43]:
has_subscribed = (y_pred >= 0.5)

In [44]:
y_val

array([0, 0, 1, ..., 0, 0, 1])

In [45]:
has_subscribed.astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [46]:
(y_val == has_subscribed).mean()

np.float64(0.8900685689006856)

In [407]:
{'all' : 0.8900685689006856,
 'age' : 0.8899579738995798,
 'balance': 0.8890732138907321,
 'marital': 0.8895155938951559,
 'previous': 0.88940499889405}

{'all': 0.8900685689006856,
 'age': 0.8899579738995798,
 'balance': 0.8890732138907321}

In [429]:
diffs = [0.8900685689006856 - 0.8899579738995798, 0.8900685689006856 - 0.8890732138907321, 
0.8900685689006856 - 0.8895155938951559, 0.8900685689006856 - 0.88940499889405]

In [431]:
diffs.sort()

In [432]:
diffs

[0.00011059500110588427,
 0.0005529750055297544,
 0.0006635700066356387,
 0.0009953550099535136]

In [503]:
diffs = [abs(0.8900685689006856 - 0.8899579738995798), abs(0.8900685689006856 - 0.88940499889405), 
abs(0.8900685689006856 - 0.8902897589028976), abs(0.8900685689006856 - 0.8906215439062154)]

In [505]:
diffs.sort()

In [506]:
diffs

[0.00011059500110588427,
 0.0002211900022119906,
 0.0005529750055297544,
 0.0006635700066356387]

In [238]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = has_subscribed.astype(int)
df_pred['actual'] = y_val
df_pred['correct'] = df_pred['prediction'] == df_pred['actual']

In [239]:
df_pred.correct.mean().round(2)

np.float64(0.88)

### Question 6

In [101]:
model = LogisticRegression(solver='liblinear', C=0.1, max_iter=1000, random_state=42)

In [102]:
model.fit(X_train, y_train)

LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear')

In [103]:
y_pred = model.predict_proba(X_val)[:, 1]

In [104]:
has_subscribed = (y_pred >= 0.5)

In [105]:
y_val

array([0, 0, 1, ..., 0, 0, 1])

In [106]:
has_subscribed.astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [107]:
(y_val == has_subscribed).mean().round(3)

np.float64(0.89)

In [108]:
len(y_val)

9042

In [109]:
len(has_subscribed)

9042

In [110]:
b = y_val == has_subscribed

In [111]:
b

array([ True,  True, False, ...,  True,  True, False])

In [112]:
import numpy as np

In [113]:
np.unique(b, return_counts=True)

(array([False,  True]), array([ 993, 8049]))

In [114]:
t = (100*8049) / 9042
t

89.01791639017917

In [115]:
f = (100*993) / 9042
f

10.982083609820837

In [100]:
88.9294403892944 + 11.070559610705596

100.0